In [6]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
import matplotlib.pyplot as plt

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


In [7]:
data_dir = r"C:\Users\Anton\OneDrive\Desktop\КПІлабки\lab3 Габунія А.П. ФІ-51мн"  # шлях до тієї папки, де ти розпакував

train_dir = os.path.join(data_dir, "seg_train", "seg_train")
val_dir   = os.path.join(data_dir, "seg_test",  "seg_test")

train_tf = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

val_tf = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_ds = ImageFolder(train_dir, transform=train_tf)
val_ds   = ImageFolder(val_dir,   transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False)

num_classes = len(train_ds.classes)
print("Класи:", train_ds.classes)
print("Кількість класів:", num_classes)
print("Train size:", len(train_ds), "Val size:", len(val_ds))


Класи: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
Кількість класів: 6
Train size: 14034 Val size: 3000


In [8]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 64 -> 32

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 32 -> 16

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU()
            # тут не зменшуємо простір, H=W=16
        )

        # 🤖 Автоматично рахуємо вихідний розмір
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 64, 64)
            feat = self.features(dummy)
            flat_dim = feat.view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Linear(flat_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model_simple = SimpleCNN(num_classes)
print(model_simple)


SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=16384, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=6, bias=True)
  )
)


In [9]:
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses, val_losses = [], []

    for epoch in range(epochs):
        # --- TRAIN ---
        model.train()
        total_loss = 0.0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train = total_loss / len(train_loader)

        # --- VAL ---
        model.eval()
        total_val = 0.0
        correct, total = 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                logits = model(xb)
                loss = criterion(logits, yb)
                total_val += loss.item()

                preds = logits.argmax(dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)

        avg_val = total_val / len(val_loader)
        acc = correct / total * 100

        train_losses.append(avg_train)
        val_losses.append(avg_val)

        print(f"[{epoch+1:02d}] train_loss={avg_train:.4f} | val_loss={avg_val:.4f} | val_acc={acc:.2f}%")

    return train_losses, val_losses


In [10]:
simple_train_loss, simple_val_loss = train_model(
    model_simple, train_loader, val_loader,
    epochs=5, lr=1e-3
)


[01] train_loss=1.0409 | val_loss=0.8255 | val_acc=69.07%
[02] train_loss=0.7652 | val_loss=0.6955 | val_acc=74.30%
[03] train_loss=0.6594 | val_loss=0.6658 | val_acc=75.07%
[04] train_loss=0.5880 | val_loss=0.5981 | val_acc=78.13%
[05] train_loss=0.5315 | val_loss=0.6063 | val_acc=77.80%
